In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [2]:
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [3]:
def e_metrics(x1, x2):
    distance = np.sum(np.square(x1 - x2))
    return np.sqrt(distance)

In [4]:
def knn(x_train, y_train, x_test, k):
    
    answers = []
    for x in x_test:
        q = 0.5
        # делаем вектор расстояний до каждой точки
        dist = np.vectorize(lambda x2: -q**e_metrics(x, x2), signature='(n)->()')(x_train)
        # соединяем вектор и класс
        dist_to_nei = np.column_stack((dist, y_train))
        # находим ближайших соседей
        nearest_nei = dist_to_nei[dist_to_nei[:,0].argsort()][:k]
        # определяем кто встречается чаще всех
        nei_freq = np.column_stack(np.unique(nearest_nei[:, -1], return_counts=True))
        # получаем самого часто встречающегося соседа
        most_freq_nei = int(nei_freq[nei_freq[:,1].argsort()[-1]][0])
        # записываем ответ
        answers.append(most_freq_nei)
        
    return answers

In [6]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [7]:
iris = load_iris()
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)

In [8]:
def pca(x_test):
    # Для начала отмасштабируем выборку
    x_new = x_test.astype(float)
    x_new = standard_scale(x_new)
    
    # Найдем собственные векторы и собственные значения
    covariance_matrix = x_new.T @ x_new
    eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

    # сформируем список кортежей (собственное значение, собственный вектор)
    eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

    # и отсортируем список по убыванию собственных значений
    eig_pairs.sort(key=lambda x: x[0], reverse=True)
    
    # Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
    W = np.hstack([eig_pairs[i][1].reshape(4, 1) for i in range(2)])

    # Сформируем новую матрицу "объекты-признаки"
    Z = x_new.dot(W)
    
    return Z

### Задача 1
Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [9]:
k = 5

In [10]:
y_pred = knn(X_train, y_train, X_test, k)
print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 5: 0.967


In [11]:
X_test2 = pca(X_test)
X_train2 = pca(X_train)

In [12]:
y_pred = knn(X_train2, y_train, X_test2, k)
print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 5: 0.867


### Вывод
При использовании PCA качества классификации снижается

### Задача 2
*Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()